In [1]:
from icecube import icetray, dataio, dataclasses, simclasses, phys_services, recclasses
import os, sys
import numpy as np
import matplotlib as mpl
from matplotlib.colors import LogNorm, Normalize
import matplotlib.pyplot as plt
import timeit as time
import math
from datetime import datetime

In [2]:
# Added by JP
# Define the directory of your files
files_dir = '/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999'
# List the contents of the entire directory
file_list_aux = os.listdir(files_dir)
# Only keep those that are I3 files
file_list = [x for x in file_list_aux if '.i3.bz2' in x]
print('Total files', len(file_list))

('Total files', 966)


In [4]:
infile = dataio.I3File(os.path.join(files_dir,file_list[0]))

In [10]:
frame = infile.pop_physics()

In [11]:
print frame

[ I3Frame  (Physics):
  'AnalysisMuons' [Physics] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (174)
  'BackgroundCLSim_intermediatePhotons' [DAQ] ==> I3Map<ModuleKey, I3Vector<I3Photon> > (3167)
  'BackgroundI3MCPESeriesMap_0.990' [DAQ] ==> I3Map<OMKey, vector<I3MCPE> > (90)
  'BackgroundI3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (1786)
  'BackgroundI3MCTree_preMuonProp' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (918)
  'BackgroundMMCTrackList' [DAQ] ==> I3Vector<I3MMCTrack> (304)
  'BeaconLaunches' [DAQ] ==> I3Map<OMKey, vector<I3DOMLaunch> > (46)
  'CalibratedWaveformRange' [DAQ] ==> I3TimeWindow (48)
  'CascadeFilter_13' [Physics] ==> I3PODHolder<bool> (27)
  'CorsikaInteractionHeight' [DAQ] ==> I3PODHolder<double> (36)
  'CorsikaWeightMap' [DAQ] ==> I3Map<string, double> (482)
  'DCAnalysisHits' [Physics] ==> I3PODHolder<double> (36)
  'DCNHits' [Physics] ==> 

In [12]:
pulses = frame['SRTInIcePulsesDOMeff'].apply(frame)

In [13]:
print pulses

[OMKey(26,55,0) => [[I3RecoPulse:
             Time : 13869
           Charge : 0.825
            Width : 8
            Flags : FADC 
]],
OMKey(36,53,0) => [[I3RecoPulse:
             Time : 12625
           Charge : 1.175
            Width : 8
            Flags : FADC 
]],
OMKey(36,57,0) => [[I3RecoPulse:
             Time : 12789
           Charge : 0.875
            Width : 1
            Flags : LC ATWD FADC 
]],
OMKey(36,59,0) => [[I3RecoPulse:
             Time : 12678
           Charge : 0.825
            Width : 1
            Flags : LC ATWD FADC 
]],
OMKey(45,40,0) => [[I3RecoPulse:
             Time : 11780
           Charge : 0.275
            Width : 8
            Flags : FADC 
]],
OMKey(45,47,0) => [[I3RecoPulse:
             Time : 12053
           Charge : 1.475
            Width : 1
            Flags : LC ATWD FADC 
]],
OMKey(45,49,0) => [[I3RecoPulse:
             Time : 12582
           Charge : 1.175
            Width : 1
            Flags : LC ATWD FADC 
]],
OMKey(45

In [16]:
one_dom = pulses.popitem()

In [17]:
one_dom

(OMKey(26,55,0), [<icecube.dataclasses.I3RecoPulse object at 0x7f6067f5a0c0>])

In [26]:
charge = 0
for pulse in one_dom[1]:
    charge += pulse.charge
print charge

0.824999988079


In [7]:
gcd_file = '/cvmfs/icecube.opensciencegrid.org/data/GCD/GeoCalibDetectorStatus_IC86_Merged.i3.gz'

gfile = dataio.I3File(gcd_file)

In [8]:
gframe = gfile.pop_frame()

In [9]:
print gframe

[ I3Frame  (Geometry):
  'I3Geometry' [Geometry] ==> I3Geometry (641528)
]



In [18]:
geometry = gframe['I3Geometry']

In [21]:
mydom = geometry.omgeo[one_dom[0]]

In [23]:
print mydom

[I3OMGeo Position: I3Position(1.71,-150.63,-421.87)
      Orientation: I3Orientation: 
    Dir: (0,0,-1)
     Up: (1,0,0)
  Right: (0,-1,0)
           OMType: 20
             Area: 0.0444 ]


In [27]:
mmctracks = frame['MMCTrackList']

In [29]:
track = mmctracks[0].particle

In [30]:
print track

[ I3Particle MajorID : 13469127016857373990
             MinorID : 846
              Zenith : 0.400847
             Azimuth : 1.80333
                   X : -227.484
                   Y : 1017.22
                   Z : 1949.99
                Time : 3536.68
              Energy : 1054.01
               Speed : 0.299792
              Length : 3125.59
                Type : MuPlus
        PDG encoding : -13
               Shape : StartingTrack
              Status : NotSet
            Location : InIce
]


In [33]:
distance = phys_services.I3Calculator.cherenkov_distance(track, mydom.position)

In [38]:
corrected_charge = charge*distance
# This is for adding also the attenuation (absorption + scattering)
att_length = 50. # Typical attenuation length in icecube
corrected_charge = charge*distance*np.exp(distance/att_length)

In [39]:
print corrected_charge

48372.61522405762


In [ ]:
# Try to put it inside a function so that you can pass the puse series to the function, 
# and obtain a list of corrected charges
# Consider looking only at DOMs "nearby" (within a maximum distance of 150-200m, no further)

In [ ]:
phys_services.I3Calculator.